<a href="https://colab.research.google.com/github/yjs000/seoul-price-data/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%EB%AC%BC%EA%B0%80_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울시 동별 물가 분석하기

사용 데이터 : [서울 열린 데이터 광장 - 서울시 개인서비스](http://data.seoul.go.kr/dataList/OA-1169/S/1/datasetView.do )  

# 데이터 전처리


1.   구별 품목별 평균 가격으로 나타냄.
2.   null, 이상치 처리

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt

from google.colab import drive

In [ ]:
price_raw = pd.read_csv("price.csv", encoding="euc-kr")
price_raw.head()

,업소일련번호,업소명,동명,주소,면적(㎡),전화번호,업종,품목코드,품목,가격(원),점검일자,구명
0,1418890727446,백영세탁,잠실동,서울특별시 송파구 올림픽로10길 58 (잠실동),NaN,02-423-2202,세탁업,1418622761392,양복 세탁료,5000.0,2021-03-25 11:13:35.0,송파구
1,1418890727332,텔미노래방,잠실동,서울특별시 송파구 올림픽로12길 5 (잠실동) - (B1호),NaN,02-414-6838,기타서비스,1418622761391,노래방 이용료,20000.0,2021-03-25 11:12:36.0,송파구
2,1418890643563,WYNN호탤,잠실동,서울특별시 송파구 올림픽로12길 5-21 (잠실동),NaN,02-420-0216,숙박업,1418622761406,숙박료 (여관),50000.0,2021-03-25 11:12:08.0,송파구
3,1418890727302,PLUS모텔,잠실동,서울특별시 송파구 올림픽로12길 5-25 (잠실동),NaN,418-4276,숙박업,1418622761406,숙박료 (여관),40000.0,2021-03-25 11:11:17.0,송파구
4,1418890674385,모텔 로즈마리,잠실동,서울특별시 송파구 올림픽로12길 5-29 (잠실동),NaN,425-1457,숙박업,1418622761406,숙박료 (여관),50000.0,2021-03-25 11:09:54.0,송파구


In [ ]:
# 구별 품목을 나타내고 싶음

grouped5 = price_raw.groupby(['주소','품목',], as_index=False).mean()
grouped5 = grouped5.drop(columns=['업소일련번호','면적(㎡)','품목코드'])
grouped5

,주소,품목,가격(원)
0,서울특별시 강남구 강남대로92길 15 (역삼동),숙박료 (여관),70000.0
1,서울특별시 강남구 개포로 307 (개포동 우성3차아파트),이용료(커트),12000.0
2,서울특별시 강남구 광평로 280 (수서동) 지하2,미용료 (커트),23000.0
3,서울특별시 강남구 광평로 280 (수서동) 지하2,미용료 (파마),80000.0
4,서울특별시 강남구 남부순환로 2941 (대치동),목욕료 (성인),8000.0
...,...,...,...
6331,서울특별시 중랑구 용마산로 623 지층 (신내동),목욕료 (성인),9000.0
6332,서울특별시 중랑구 용마산로 623 지층 (신내동),찜질방이용료,9000.0
6333,서울특별시 중랑구 중랑역로 75 (중화동),생선초밥,10000.0
6334,서울특별시 중랑구 중랑역로 86-1 (중화동),냉면(물),6000.0


In [ ]:
# grouped5에 구 열 추가
address = grouped5['주소']
gu = []
for i in address:
  tokens = i.split() # 구를 구하기 위해 split
  gu.append(tokens[1])

grouped5['구'] = gu
grouped5

,주소,품목,가격(원),구
0,서울특별시 강남구 강남대로92길 15 (역삼동),숙박료 (여관),70000.0,강남구
1,서울특별시 강남구 개포로 307 (개포동 우성3차아파트),이용료(커트),12000.0,강남구
2,서울특별시 강남구 광평로 280 (수서동) 지하2,미용료 (커트),23000.0,강남구
3,서울특별시 강남구 광평로 280 (수서동) 지하2,미용료 (파마),80000.0,강남구
4,서울특별시 강남구 남부순환로 2941 (대치동),목욕료 (성인),8000.0,강남구
...,...,...,...,...
6331,서울특별시 중랑구 용마산로 623 지층 (신내동),목욕료 (성인),9000.0,중랑구
6332,서울특별시 중랑구 용마산로 623 지층 (신내동),찜질방이용료,9000.0,중랑구
6333,서울특별시 중랑구 중랑역로 75 (중화동),생선초밥,10000.0,중랑구
6334,서울특별시 중랑구 중랑역로 86-1 (중화동),냉면(물),6000.0,중랑구


In [ ]:
#구별 품목의 평균을 구함

# grouped5.get('구', default='ㅁ')
grouped5 = grouped5.groupby(['구','품목'],as_index=False).mean()
grouped5

,구,품목,가격(원)
0,강남구,김밥,3000.000000
1,강남구,다방 국산차(녹차),5000.000000
2,강남구,돈가스,12000.000000
3,강남구,목욕료 (성인),8000.000000
4,강남구,미용료 (커트),20000.000000
...,...,...,...
653,중랑구,찜질방이용료,9000.000000
654,중랑구,치킨,15000.000000
655,중랑구,커피(외식),2900.000000
656,중랑구,탕수육,14500.000000


In [ ]:
#null 값 확인 후 처리

grouped5 = grouped5.dropna()
grouped5.isnull().sum().sum()

# 품목이 해당없음인걸 뺌
notNa = grouped5['품목'] != '해당없음'
grouped5 = grouped5[notNa]
grouped5.isnull().sum().sum()

#공동주택관리비, 골프연습장 이용료는 개수도 적고, 가격이 너무 비싸서 물가를 구하기 적절하지 않음 -> 뺌

is_golf = grouped5['품목'] == '골프연습장 이용료'
is_fee = grouped5['품목'] == '공동주택관리비'
isGolf = grouped5['품목'].isin(['골프연습장 이용료', '공동주택관리비'])
grouped5 = grouped5[~isGolf]
grouped5

,구,품목,가격(원)
0,강남구,김밥,3000.000000
1,강남구,다방 국산차(녹차),5000.000000
2,강남구,돈가스,12000.000000
3,강남구,목욕료 (성인),8000.000000
4,강남구,미용료 (커트),20000.000000
...,...,...,...
653,중랑구,찜질방이용료,9000.000000
654,중랑구,치킨,15000.000000
655,중랑구,커피(외식),2900.000000
656,중랑구,탕수육,14500.000000


# 공통 품목 확인

1.   구별 공통 품목만 남김


In [ ]:
# 구의 개수 확인 # 검색으로 서울시 전체 구가 25개임을 확인함
a = grouped5.groupby(['구']).mean()
a.count()

가격(원)    25
dtype: int64

In [ ]:
# 각 품목이 몇개의 구에 있는지 확인 
freq = grouped5['품목'].value_counts() 
freq = freq.to_dict()
freq

{'PC방 이용료': 4,
 '갈비탕': 12,
 '기타': 3,
 '김밥': 14,
 '김치찌개 백반': 19,
 '냉면(물)': 17,
 '노래방 이용료': 5,
 '다방 국산차(녹차)': 20,
 '당구장 이용료': 5,
 '돈가스': 20,
 '돼지갈비(외식)': 11,
 '된장찌개 백반': 20,
 '목욕료 (성인)': 25,
 '미용료 (커트)': 25,
 '미용료 (파마)': 25,
 '볼링장 이용료': 1,
 '불고기': 10,
 '비빔밥': 16,
 '사진 인화료': 6,
 '사진 촬영료': 6,
 '삼겹살': 18,
 '삼계탕': 14,
 '생선초밥': 22,
 '설렁탕': 11,
 '쇠고기(외식)': 10,
 '수영장 입장료': 4,
 '숙박료 (여관)': 24,
 '숙박료 (호텔)': 6,
 '양복 세탁료': 24,
 '영화 관람료': 19,
 '의복수선료': 18,
 '이용료(커트)': 25,
 '자장면': 25,
 '조리라면': 16,
 '짬뽕': 25,
 '찜질방이용료': 12,
 '치킨': 17,
 '칼국수': 9,
 '커피(외식)': 25,
 '탕수육': 25,
 '택배이용료': 2,
 '피자': 22,
 '햄버거': 11}

In [ ]:
# 모든 구에 있는 품목
all = []

for key, value in freq.items():
  if (value == 25):
    all.append(key)

all

['미용료 (파마)', '커피(외식)', '목욕료 (성인)', '짬뽕', '탕수육', '미용료 (커트)', '이용료(커트)', '자장면']

In [ ]:
df = grouped5[grouped5['품목'].isin(['미용료 (커트)', '짬뽕', '목욕료 (성인)', '자장면', '커피(외식)', '탕수육', '이용료(커트)', '미용료 (파마)'])]
df

,구,품목,가격(원)
3,강남구,목욕료 (성인),8000.0
4,강남구,미용료 (커트),20000.0
5,강남구,미용료 (파마),65000.0
11,강남구,이용료(커트),12000.0
12,강남구,자장면,6500.0
...,...,...,...
649,중랑구,이용료(커트),8000.0
650,중랑구,자장면,3500.0
652,중랑구,짬뽕,4500.0
655,중랑구,커피(외식),2900.0


# 물가 평균


1.   구별로 공통되는 품목만 남겨둠
2.   구별 평균을 구함



In [ ]:
#품목이 같기 때문에 구별 평균을 구함
df1 = df.groupby(['구']).mean()
df1

,가격(원)
구,
강남구,18393.750000
강동구,11270.833333
강북구,10406.250000
강서구,13697.916667
관악구,12617.439682
광진구,12972.601912
구로구,9095.833333
금천구,12229.125000
노원구,10565.625000


In [ ]:
sorted = df1['가격(원)'].sort_values()
sorted

구
구로구      9095.833333
용산구      9608.333333
강북구     10406.250000
노원구     10565.625000
은평구     10751.041667
영등포구    10754.166667
도봉구     11148.674599
강동구     11270.833333
종로구     11962.500000
동작구     12012.500000
동대문구    12053.125000
성북구     12098.958333
금천구     12229.125000
관악구     12617.439682
광진구     12972.601912
서대문구    13062.500000
중구      13362.500000
양천구     13454.166667
송파구     13630.634057
강서구     13697.916667
성동구     14256.250000
중랑구     14737.500000
서초구     16345.833333
마포구     16588.541667
강남구     18393.750000
Name: 가격(원), dtype: float64

In [ ]:
%matplotlib inline


# 품목별로 가격의 분포를 확인
data = df

# 차트 윈도우 타이틀
plt.figure()

# 차트 제목
plt.title("Data Analysis Chart")

# x, y 축 제목
plt.xlabel("품목")
plt.ylabel("가격(원)")

# # plt 적용, 한개의 화면에서 나타나도록 하기 위한 axes 설정
# ax = plt.gca()

# kind 차트 종류 - bar, line. scatter 등
data.plot(kind='scatter', x='품목', y='가격(원)', ax=None)


# 차트 보여주기
plt.show()


TypeError: ignored

# 순위평균


1.   구별 공통 품목만 남김
2.   품목별 순위를 매김
3.   구별로 매겨진 순위를 평균냄



In [ ]:
df


,구,품목,가격(원)
3,강남구,목욕료 (성인),8000.0
4,강남구,미용료 (커트),20000.0
5,강남구,미용료 (파마),65000.0
11,강남구,이용료(커트),12000.0
12,강남구,자장면,6500.0
...,...,...,...
649,중랑구,이용료(커트),8000.0
650,중랑구,자장면,3500.0
652,중랑구,짬뽕,4500.0
655,중랑구,커피(외식),2900.0


In [ ]:
# 구별 품목이 커피(외식)인걸 뽑음

is_all_0 = df['품목'] == all[0]
all_0 = df[is_all_0]
all_0

,구,품목,가격(원)
5,강남구,미용료 (파마),65000.000000
26,강동구,미용료 (파마),27500.000000
50,강북구,미용료 (파마),32500.000000
82,강서구,미용료 (파마),46000.000000
118,관악구,미용료 (파마),39759.740260
159,광진구,미용료 (파마),40067.901235
194,구로구,미용료 (파마),20000.000000
213,금천구,미용료 (파마),35000.000000
230,노원구,미용료 (파마),30000.000000
261,도봉구,미용료 (파마),32612.318841


In [ ]:
# 구별 커피의 순위 # 1~25 : 1일수록 비쌈 
# 동점처리 : 평균
# https://rfriend.tistory.com/461 순위에서 동점처리하는방법 참고
all_0['rank'] = all_0['가격(원)'].rank(ascending=False)
all_0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,구,품목,가격(원),rank
5,강남구,미용료 (파마),65000.000000,1.0
26,강동구,미용료 (파마),27500.000000,23.0
50,강북구,미용료 (파마),32500.000000,19.0
82,강서구,미용료 (파마),46000.000000,6.0
118,관악구,미용료 (파마),39759.740260,12.0
159,광진구,미용료 (파마),40067.901235,9.0
194,구로구,미용료 (파마),20000.000000,24.5
213,금천구,미용료 (파마),35000.000000,14.5
230,노원구,미용료 (파마),30000.000000,21.0
261,도봉구,미용료 (파마),32612.318841,18.0


In [ ]:
# all[1]
# 구별 품목이 탕수육 인걸 뽑음

is_all_1 = df['품목'] == all[1]
all_1 = df[is_all_1]
all_1

# 구별 커피의 순위
all_1['rank'] = all_1['가격(원)'].rank(ascending=False)
all_1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,구,품목,가격(원),rank
16,강남구,커피(외식),4650.000000,1.0
38,강동구,커피(외식),3000.000000,18.0
67,강북구,커피(외식),3000.000000,18.0
100,강서구,커피(외식),3500.000000,5.5
140,관악구,커피(외식),3225.581395,11.0
182,광진구,커피(외식),3155.721393,14.0
204,구로구,커피(외식),3433.333333,7.0
222,금천구,커피(외식),2000.000000,25.0
242,노원구,커피(외식),2725.000000,23.0
284,도봉구,커피(외식),2920.000000,21.0


In [ ]:
# all[2]
# 구별 품목이 미용료 (커트) 인걸 뽑음

is_all_2 = df['품목'] == all[2]
all_2 = df[is_all_2]
all_2

# 구별 커피의 순위
all_2['rank'] = all_2['가격(원)'].rank(ascending=False)
all_2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,구,품목,가격(원),rank
3,강남구,목욕료 (성인),8000.000000,3.0
24,강동구,목욕료 (성인),7000.000000,10.5
48,강북구,목욕료 (성인),5750.000000,24.0
80,강서구,목욕료 (성인),7500.000000,4.5
116,관악구,목욕료 (성인),7000.000000,10.5
157,광진구,목욕료 (성인),7347.826087,6.0
192,구로구,목욕료 (성인),6500.000000,16.5
211,금천구,목욕료 (성인),6000.000000,21.5
228,노원구,목욕료 (성인),6000.000000,21.5
259,도봉구,목욕료 (성인),6227.272727,19.0


In [ ]:
# all[3]
# 구별 품목이 목욕료 (성인) 인걸 뽑음

is_all_3 = df['품목'] == all[3]
all_3 = df[is_all_3]
all_3

# 구별 커피의 순위
all_3['rank'] = all_3['가격(원)'].rank(ascending=False)
all_3

In [ ]:
# all[4]
# 구별 품목이 자장면 인걸 뽑음

is_all_4 = df['품목'] == all[4]
all_4 = df[is_all_4]
all_4

# 구별 커피의 순위
all_4['rank'] = all_4['가격(원)'].rank(ascending=False)
all_4

In [ ]:
# all[5]
# 구별 품목이 이용료(커트) 인걸 뽑음

is_all_5 = df['품목'] == all[5]
all_5 = df[is_all_5]
all_5

# 구별 커피의 순위
all_5['rank'] = all_5['가격(원)'].rank(ascending=False)
all_5

In [ ]:
# all[6]
# 구별 품목이 짬뽕  인걸 뽑음

is_all_6 = df['품목'] == all[6]
all_6 = df[is_all_6]
all_6

# 구별 커피의 순위
all_6['rank'] = all_6['가격(원)'].rank(ascending=False)
all_6

In [ ]:
# all[7]
# 구별 품목이 미용료 (파마)  인걸 뽑음

is_all_7 = df['품목'] == all[7]
all_7 = df[is_all_7]
all_7

# 구별 커피의 순위
all_7['rank'] = all_7['가격(원)'].rank(ascending=False)
all_7

In [ ]:
# all_0부터 7까지 합침
concat = pd.concat([all_0, all_1, all_2, all_3, all_4, all_5, all_6, all_7])
concat

In [ ]:
#구별 순위의 평균을 구함
result = concat.groupby(['구'],as_index=False).mean()
result

In [ ]:
result = result.drop('가격(원)', axis=1)
result.sort_values(by='rank', ascending=False)

In [ ]:
# 엑셀파일로 저장 전체/dataset/price_rank.xlsx
result.to_excel("price_rank.xlsx", index=False)

In [ ]:
# 동, 업종, 품목별 그룹
grouped2 = price_raw.groupby(['동명','업종','품목'],as_index=False).mean()
grouped2 = grouped2.drop(columns=['업소일련번호','면적(㎡)','품목코드'])
grouped2

In [ ]:
#null 값 확인 후 처리

# grouped2.isnull().sum().sum()
grouped2 = grouped2.dropna()
grouped2.isnull().sum().sum()

# 품목이 해당없음인걸 뺌
notNa = grouped2['품목'] != '해당없음'
grouped2 = grouped2[notNa]
grouped2

In [ ]:
# 데이터 확인
df3 = grouped2.drop(columns=['업종'])
df3

In [ ]:
import matplotlib.pyplot as plt
# 품목별로 가격의 분포를 확인
data = df3

# 차트 윈도우 타이틀
plt.figure()

# 차트 제목
plt.title("Data Analysis Chart")

# 차트 글꼴
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'font.family': 'batang'})

# x, y 축 제목
plt.xlabel("품목")
plt.ylabel("가격(원)")

# # plt 적용, 한개의 화면에서 나타나도록 하기 위한 axes 설정
# ax = plt.gca()

# kind 차트 종류 - bar, line. scatter 등
data.plot(kind='scatter', x='품목', y='가격(원)', ax=None)


# 차트 보여주기
plt.show()


In [ ]:
# 가격별로 sort
df3.sort_values(by=['가격(원)'])

#공동주택관리비, 골프연습장 이용료는 개수도 적고, 가격이 너무 비싸서 물가를 구하기 적절하지 않음 -> 뺌

is_golf = df['품목'] == '골프연습장 이용료'
is_fee = df['품목'] == '공동주택관리비'
df3 = df[~(is_golf & is_fee)]
df3

#나머지의 평균을 구함
df3.groupby(['동명']).mean()

# 전체품목관련

In [ ]:
# 동별로 몇개의 품목이 있는지 확인

df2 = df.groupby(['동명'])
df2.size()

In [ ]:
#모든 품목
#각 품목이 몇개의 동에 있는지 확인 -> 가장 많은 동에분포하는 품목을 대표 품목으로 정함
freq2 = grouped2['품목'].value_counts() 
freq2 = freq2.to_dict()
freq2

In [ ]:
# 모든 품목
# 40개 이상의 동에 존재하는 품목 list를 만듦
representative2 = []

for key, value in freq2.items():
  if (value >= 40):
    representative2.append(key)

representative2

In [ ]:
# def statement(town):
#   for i in range(len(df)-1) : 
#     row = df.iloc[i] # row를 하나씩 보면서
#     if(row['동명'] == town):
#       if row['품목'] in representative: # 그 동의 품목이 하나라도 대표품목에 들어있으면
#         return True
#       else: return False

# for town in town_array:
#   if statement(town):
#     continue #바로 다음 동으로 넘어감
#   else:
#     print(town) 
        